In [1]:
# import packages 
import numpy as np
import pandas as pd

import os

from bokeh.plotting import figure, show, output_file, save

#from bokeh.embed import components, file_html
from bokeh.io import output_notebook, show, curdoc
from bokeh.models import Panel, Tabs, Slider, CustomJS, HoverTool, ColumnDataSource, Range1d, GraphRenderer,Text,Legend
from bokeh.models import SingleIntervalTicker, LinearAxis, TableColumn, DataTable, HTMLTemplateFormatter
from bokeh.models.widgets import RangeSlider, Select, RadioGroup, MultiSelect, TextInput
from bokeh.layouts import widgetbox, gridplot
#from bokeh.resources import CDN
from bokeh.layouts import column, row, layout
from bokeh.palettes import inferno, viridis, Turbo256 # select a palette
from bokeh.models import BasicTicker, ColorBar, LinearColorMapper, ColumnDataSource, PrintfTickFormatter, Whisker
from bokeh.transform import transform
from bokeh.models.mappers import LogColorMapper
from bokeh.models import Title

from scipy.stats import shapiro 

from bokeh.colors import RGB
from matplotlib import cm

#from bokeh.palettes import Dark2_5 as palette
import itertools  
#output_notebook()

import os
from os.path import exists
import sys

from datetime import datetime as datetime_import
import time

import matplotlib.pyplot as plt

from scipy.stats import norm
from scipy.stats import anderson

import cv2
import copy

import re

from bs4 import BeautifulSoup 

pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
Version = 'v2.9'

In [ ]:
def extra_calculation(df_full):
    return df_full

In [ ]:
def Cube_calculations(df_full):
    #Calculations

    #Scaling
    X_target = ['TF','TB','DF','DB']
    Y_target = ['TL','TR','DL','DR']
    Z_target = ['FL','FR','BL','BR']
    df_full['X_scaling'] = 0
    df_full['Y_scaling'] = 0
    df_full['Z_scaling'] = 0

    for i in range(len(df_full)):
        X_scaling = np.array([])
        Y_scaling = np.array([])
        Z_scaling = np.array([])
        for T in range(len(X_target)):

            df_cube = df_full.loc[i]
            X_scaling = np.append(X_scaling, np.append( ( 
                df_cube[X_target[T]+'A']+df_cube[X_target[T]+'B'])/20, (df_cube[X_target[T]+'C']+df_cube[X_target[T]+'B'])/20))
            Y_scaling = np.append(Y_scaling, np.append( ( 
                df_cube[Y_target[T]+'A']+df_cube[Y_target[T]+'B'])/20, (df_cube[Y_target[T]+'C']+df_cube[Y_target[T]+'B'])/20))
            Z_scaling = np.append(Z_scaling, np.append( ( 
                df_cube[Z_target[T]+'A']+df_cube[Z_target[T]+'B'])/20, (df_cube[Z_target[T]+'C']+df_cube[Z_target[T]+'B'])/20))
        df_full.loc[i, 'X_scaling'] = np.mean((X_scaling-1))*100
        df_full.loc[i, 'Y_scaling'] = np.mean((Y_scaling-1))*100
        df_full.loc[i, 'Z_scaling'] = np.mean((Z_scaling-1))*100

    #FaceOffset 
    F_target = ['TF','TB','TL','TR','DF','DB','DL','DR']
    df_full['FaceOffest'] = 0
    for i in range(len(df_full)):
        FO = np.array([])
        df_cube = df_full.loc[i]
        for T in range(len(F_target)):
            FO = np.append(FO, np.append( ( 
                df_cube[F_target[T]+'A']-df_cube[F_target[T]+'B'])/4, (df_cube[F_target[T]+'C']-df_cube[F_target[T]+'B'])/4))
        df_full.loc[i, 'FaceOffest'] = -np.mean(FO)


    #ZOverCuring
    ZOC_target = ['FL','FR','BL','BR']
    df_full['ZOverCuring'] = 0
    for i in range(len(df_full)):
        ZOC = np.array([])
        df_cube = df_full.loc[i]
        for T in range(len(ZOC_target)):
            ZOC = np.append(ZOC, np.mean(np.append( ( 
                df_cube[ZOC_target[T]+'A']-df_cube[ZOC_target[T]+'B'])/4, (df_cube[ZOC_target[T]+'C']-df_cube[ZOC_target[T]+'B'])/4)))
        df_full.loc[i, 'ZOverCuring'] = -np.mean(ZOC*2)


    #Accuracy 
    F_target = ['TF','TB','TL','TR','DF','DB','DL','DR']
    df_full['Accuracy_XY'] = 0
    for i in range(len(df_full)):
        Accuracy_array = np.array([])
        for T in range(len(F_target)):
            df_cube = df_full.loc[i]

            Accuracy_array = np.append(Accuracy_array,  [df_cube[F_target[T]+'A'],df_cube[F_target[T]+'B'],df_cube[F_target[T]+'C']] )
        Deviation = np.mean(np.mean(Accuracy_array-10))
        Std = np.std(Accuracy_array, ddof=1) #sample std. Without ddof=1 population std
        Accuracy = abs(Deviation) + 2*Std 
        df_full.loc[i, 'Accuracy_XY'] = np.mean(Accuracy)

    #Accuracy 
    F_target = ['FL','FR','BL','BR','TF','TB','TL','TR','DF','DB','DL','DR']
    df_full['Accuracy'] = 0
    for i in range(len(df_full)):
        Accuracy_array = np.array([])
        for T in range(len(F_target)):
            df_cube = df_full.loc[i]

            Accuracy_array = np.append(Accuracy_array,  [df_cube[F_target[T]+'A'],df_cube[F_target[T]+'B'],df_cube[F_target[T]+'C']] )
        Deviation = np.mean(np.mean(Accuracy_array-10))
        Std = np.std(Accuracy_array, ddof=1) #sample std. Without ddof=1 population std
        Accuracy = abs(Deviation) + 2*Std 
        df_full.loc[i, 'Accuracy'] = np.mean(Accuracy)

    # Shifting
    df_full['ShiftingXZ'] = 0
    df_full['ShiftingYZ'] = 0
    df_full['ShiftingXY'] = 0
    for i in range(len(df_full)):
        df_cube = df_full.loc[i]
        ShiftingXZ = np.mean((90-df_cube['Angle DL'] + df_cube['Angle DR']-90 + df_cube['Angle TL']-90 + 90-df_cube['Angle TR'])/4)
        ShiftingYZ = np.mean((df_cube['Angle TF']-90 + 90-df_cube['Angle TB'] + 90-df_cube['Angle DF'] + df_cube['Angle DB']-90)/4)
        ShiftingXY = np.mean((df_cube['Angle BL']-90 + 90-df_cube['Angle FL'] + 90-df_cube['Angle BR'] + df_cube['Angle FR']-90)/4)
        df_full.loc[i, 'ShiftingXZ'] = ShiftingXZ
        df_full.loc[i, 'ShiftingYZ'] = ShiftingYZ
        df_full.loc[i, 'ShiftingXY'] = ShiftingXY
        
    return df_full

In [ ]:
def HoReRe_calculation(df_full):
    
    thickness_CAD = 8
    width_CAD = 8
    Knob_CAD = 5
    
    #Target parameters
    #Shifting
    df_full['XZ_Shear'] = np.mean( [-(df_full['Angle_D_F']-90) + (df_full['Angle_C_D']-90) - (df_full['Angle_A_C']-90) +(df_full['Angle_A_F']-90)],axis=0 )
    df_full['YZ_Shear'] = np.mean( [-(df_full['Angle_E_F']-90) + (df_full['Angle_B_F']-90) - (df_full['Angle_B_C']-90) +(df_full['Angle_C_E']-90)],axis=0 )
    df_full['XY_Shear'] = np.mean( [-(df_full['Angle_D_E']-90) + (df_full['Angle_A_E']-90) - (df_full['Angle_A_B']-90) +(df_full['Angle_B_D']-90)],axis=0 )


    #Calculate Face_offset (Only for HoReRe cubes)
    df_full['Face_offset_X'] = (np.mean([df_full['BFU34']-thickness_CAD,df_full['BFU12']-thickness_CAD],axis=0) - (df_full['BFU23']-width_CAD)  )/4
    df_full['Face_offset_Y'] = (np.mean([df_full['AFV12']-thickness_CAD,df_full['AFV34']-thickness_CAD],axis=0) - (df_full['AFV23']-width_CAD)  )/4
    df_full['Face_offset_Z'] = (np.mean([df_full['AEW12']-thickness_CAD,df_full['AEW34']-thickness_CAD],axis=0) - (df_full['AEW23']-width_CAD)  )/4

    df_full['Face_offset_XY'] = np.mean([df_full['Face_offset_X'],df_full['Face_offset_Y']],axis=0)

    df_full['Face_offset_XY_diameter'] = (df_full['ABFW_diameter']-Knob_CAD)/2


    #Calculate Scaling (Only for HoReRe cubes)
    df_full['Scaling_X_pct'] = (np.mean([ df_full['BFU12']+df_full['BFU23'], df_full['BFU23']+df_full['BFU34']],axis=0) / (thickness_CAD+width_CAD)-1)*100
    df_full['Scaling_Y_pct'] = (np.mean([ df_full['AFV12']+df_full['AFV23'], df_full['AFV23']+df_full['AFV34']],axis=0) / (thickness_CAD+width_CAD)-1)*100
    df_full['Scaling_Z_pct'] = (np.mean([ df_full['AEW12']+df_full['AEW23'], df_full['AEW23']+df_full['AEW34']],axis=0) / (thickness_CAD+width_CAD)-1)*100


    df_full['Scaling_X_noW_pct'] = (np.mean([ df_full['BFU12'], df_full['BFU34']],axis=0) / (thickness_CAD)-1)*100
    df_full['Scaling_Y_noW_pct'] = (np.mean([ df_full['AFV12'], df_full['AFV34']],axis=0) / (thickness_CAD)-1)*100
    df_full['Scaling_Z_noW_pct'] = (np.mean([ df_full['AEW12'], df_full['AEW34']],axis=0) / (thickness_CAD)-1)*100


    df_full['Z_compensation'] = df_full['Face_offset_Z']*2

    df_full['Z_compensation_diameter'] = abs(np.mean([df_full['BCDU_diameter'],df_full['ACEV_diameter']],axis=0) - df_full['ABFW_diameter'])
    df_full['Z_compensation_roundness'] = abs(np.mean([df_full['BCDU_Roundness'],df_full['ACEV_Roundness']],axis=0) - df_full['ABFW_Roundness'])

   
    
    return df_full

In [ ]:
def Carl_caluclation(df_full):
    #Target parameters
    M500A_CAD = 3
    
    
    df_full['Scaling_X_pct'] = (df_full['M500A']/M500A_CAD - 1) * 100
    
    return df_full

In [1]:
def make_file(Path,Data_folder,Save_folder,Version,PartID_loc):
    
    

    

    
    folder = Data_folder

    if os.path.exists(Save_folder)==False:
        os.mkdir(Save_folder) 


    df_full = pd.DataFrame({})





    for Folder in os.listdir(folder):
        if Folder[0]!='.':
            date = Folder.split(' ')[-1]
            date = datetime_import.strptime(date,'%d%m%Y')
            machine = Folder.split(' ')[-2]
            job = Folder.split(' ')[-3]
            df_all = pd.DataFrame({})



            for file in os.listdir(folder+Folder):

                if file.endswith('xls') | file.endswith('xlsx'):
                    df = pd.read_excel(folder+Folder+'/'+file,skiprows=12)
                    name = file.split('.')[0].split('_')[-1]
                    df = df.rename(columns = {'Characteristic':'index'})

                    df_flip = pd.DataFrame([np.asarray(df['Actual'])],columns=df['index'])
                    df_flip['Date'] = date
                    df_flip['Machine'] = machine
                    df_flip.insert(0, 'Machine', df_flip.pop('Machine'))
                    df_flip['Print_ID'] = job
                    df_flip.insert(0, 'Print_ID', df_flip.pop('Print_ID'))
                    df_flip['Cube'] = name
                    df_flip.insert(0, 'Cube', df_flip.pop('Cube'))    
                    
                    
                    part = (int(name[-1])-1)*9 + ( ord(name[0]) - 64 )
                    
                    df_flip.insert(0, 'Part_number', part ) 

                    df_all = pd.concat([df_all,df_flip])
                    
            df_all = df_all.reset_index(inplace = False,drop=True) 
            

            
            
            #df_all = extra_calculation(df_all)

            df_full = pd.concat([df_all,df_full])
            
            
            
    # Calibration cubes
    Calibration_cubes = True
    if Calibration_cubes: 
        Layout_file = 'JB266_Unscaled.openjob'
        extra_calculations = extra_calculation
        
        part_indx = 0
    
    
    Openjob_folder = 'openjob'
    with open(Openjob_folder+'/'+Layout_file, 'r') as f:
        Layout_data = f.read() 

    # Passing the stored data inside the beautifulsoup parser 
    bs_data = BeautifulSoup(Layout_data, 'xml') 

    # Finding all instances of tag   
    b_unique = bs_data.find_all('part') 

    Layout = pd.DataFrame({})

    for i in range(0,len(b_unique)):
 
        part = b_unique[i]["name"].split('_')[part_indx]
        part = part.split('.')[0]
    

        xcoor_build = np.float64(b_unique[i].translation.x.string)
        ycoor_build = np.float64(b_unique[i].translation.y.string)
        zcoor_build = np.float64(b_unique[i].translation.z.string)


        Layout = pd.concat([Layout,pd.DataFrame({'Part_number': [np.int64(part)],'X_coord':[xcoor_build],'Y_coord':[ycoor_build],'Z_coord':[zcoor_build]})])
    Layout = Layout.reset_index(drop=True)


    df_full = df_full.merge(Layout, on = 'Part_number', how = 'inner')  


    Layout_layers = pd.DataFrame({'Layer': np.unique(Layout['Z_coord'])})



    #----Layout End----
    df_full = df_full.reset_index(drop=True)
    
    df_full = Cube_calculations(df_full)

    df_full['Layer'] = 9999    
    for layers in range(len(Layout_layers)):
        df_full['Layer'][df_full['Z_coord'] == Layout_layers['Layer'][layers]] = layers+1

    df_full['Galvo'] = 1  
    


    df_full['Print_ID_layer'] = df_full['Print_ID'] + '_' + df_full['Layer'].astype(str)

    df_full['Print_ID_galvo'] = df_full['Print_ID'] + '_' + df_full['Galvo'].astype(str)
    
    
    df_full['Version'] = Version
    
    df_full.columns = df_full.columns.str.replace(' ', '_')
    
    df_full.to_csv(Save_folder+"/all_combined_values.csv", index=False)
        
        
        
    return df_full



In [2]:
#Plotting of heatmapes
def heatmaps(data,parameters,job,Colourmap_range,method,coolwarm_palette,plots,Visual):
    
    
    IDs = data[['Part_number','Print_ID','Print_ID_layer','Print_ID_galvo','Layer','X_coord','Y_coord']] #remember add galvo
    
    data_dropped = data.drop(columns=['Part_number','Print_ID','Print_ID_layer','Print_ID_galvo','Layer','Version']) #remember add galvo
    
    #Plot mean, std or result values in heatmap
    if method == 'Mean':
        
        df_sub = data_dropped.groupby(['X_coord','Y_coord']).mean(numeric_only=True).reset_index()

        df_sub = df_sub.merge(IDs, on = ['X_coord','Y_coord'])
        
        df_sub = df_sub[df_sub['Print_ID'] == np.unique(df_sub['Print_ID'])[0]] 
        
    elif method == 'Std':
            
        df_sub = data_dropped.groupby(['X_coord','Y_coord']).std().reset_index()
        
        df_sub = df_sub.merge(IDs, on = ['X_coord','Y_coord'])
        
        df_sub = df_sub[df_sub['Print_ID'] == np.unique(df_sub['Print_ID'])[0]] 
        
        
    for parameter in parameters:

        #If more layers, split based on layer
        if method == 'Original':
            if 'Print_ID_layer' in data.columns:
                df_sub = data[data['Print_ID_layer']==job]
            else:
                df_sub = data[data['Print_ID']==job]

                
        #If only one job table becomes NaN. Replace with 0. 
        if np.all(np.isnan(df_sub[parameter].values)):
            df_sub = df_sub.replace(np.nan, 0)
        df_sub[parameter+'_mini'] = df_sub[parameter].round(2)



        #Define range for colormap   
        if method == 'Std':
                lower = df_sub[parameter].min()
                upper = df_sub[parameter].max()
        else:
            if parameter+'_lower' in Colourmap_range:
                lower = Colourmap_range[parameter+'_lower'].values[0]
                upper = Colourmap_range[parameter+'_upper'].values[0]
            else:
                lower = data[parameter].min()
                upper = data[parameter].max()
            
            

        mapper = LinearColorMapper(
        palette=coolwarm_palette, low=lower, high=upper)

        source = ColumnDataSource(df_sub )
        

        #Make heatmap plot
        pixel_x = np.int64(df_sub["Y_coord"].max()+np.median(np.diff(np.sort(np.unique(df_sub["Y_coord"])))))
        pixel_y =  np.int64(df_sub["X_coord"].max()+np.median(np.diff(np.sort(np.unique(df_sub["X_coord"])))))
        pixel_factor = 2
        
        x_range = Range1d(start = 0, end = pixel_y)
        y_range = Range1d(start = 0, end = pixel_x)

        plot_width = 800
        plot_height = 400
        
        p = figure(
            plot_width=plot_width,
            plot_height=plot_height,
            title=parameter,
            #x_axis_type=None,
            #y_axis_type=None,
            x_range=x_range, 
            y_range=y_range
        )
        
        p.add_layout(Title(text=data['Version'].iloc[-1], align="right",text_color ='grey'), "right")
        

        
        x_offset = np.int64(df_sub["X_coord"].min()/2)
        y_offset = np.int64(df_sub["Y_coord"].min()/2)
        
        
        if Visual == 'Geometry' and not os.path.isdir('Geometry'):# (image_file != None): 
            print('Make folder called Geometry with a png file of your geometry if you want pictures. \
                      Defaulting to regtangles')
            
            print(len(df_sub)>1)
            print(df_sub)
            if len(df_sub)>1:
                width = np.int64(np.min(np.diff(np.sort(np.unique(df_sub["X_coord"])))))
                height = np.int64(np.min(np.diff(np.sort(np.unique(df_sub["Y_coord"])))))
            else:
                width = 10
                height = 10
            
            p.rect(
                x="X_coord",
                y="Y_coord",
                width=width,
                height=height,
                source=source,
                line_color='black',
                line_width = 0.5,
                fill_color=transform(parameter, mapper))
        elif Visual == 'Geometry' and os.path.isdir('Geometry'):# and (image_file != None):

            

            image_file = os.listdir('Geometry')[0]
            
            image = cv2.imread('Geometry/'+ image_file,cv2.IMREAD_GRAYSCALE)
            image = np.asarray(image).astype('float').round(2)
               
            
            
            im_h = np.int64(np.floor(np.shape(image)[1]/10))
            im_w = np.int64(np.floor(np.shape(image)[0]/10))
            
            dw = np.int64(np.min(np.diff(np.sort(np.unique(df_sub["X_coord"]))))* 0.9) * pixel_factor
            dh = np.int64(dw / im_w * im_h *plot_height/plot_width)
            
            
            image = cv2.resize(image, (dw, dh),
               interpolation = cv2.INTER_AREA)
            image = np.flipud(np.ceil(image))
        
            
            
            
            background = np.ones((pixel_x*pixel_factor,pixel_y*pixel_factor)) * 255 
            
            for h in range(len(df_sub)):
                corner_x = np.int64(df_sub["X_coord"].iloc[h]*pixel_factor) - x_offset *2
                corner_y = np.int64(df_sub["Y_coord"].iloc[h]*pixel_factor) - y_offset*2
                

                image_c = copy.deepcopy(image)
                
                image_c[image_c < 250] = df_sub[parameter].iloc[h]
                image_c[image_c >= 250] = np.mean([upper,lower])
                

                background[corner_y:corner_y+np.shape(image_c)[0],corner_x:corner_x+np.shape(image_c)[1]] = image_c
                
            background[background >= 250] = np.mean([upper,lower])
            
            
            p.image(image=[background],  dh=pixel_x, dw=pixel_y, x=0, y=0, color_mapper = mapper)
#        
        elif Visual == 'Surface':
        
            nx = 100
            ny = 100

            xi = np.linspace(df_sub["X_coord"].min(), df_sub["X_coord"].max(), nx)
            yi = np.linspace(df_sub["Y_coord"].min(), df_sub["Y_coord"].max(), ny)
            xi, yi = np.meshgrid(xi, yi)

            zi = griddata( (df_sub["X_coord"],df_sub["Y_coord"]), df_sub[parameter], (xi,yi), method = 'linear' )


            dh = 320
            dw = 480

            p.image(image=[zi], dh=dh, dw=dw, x=0, y=0, color_mapper = mapper) 
            
        else: 
            if len(df_sub)>1:
                width = np.int64(np.min(np.diff(np.sort(np.unique(df_sub["X_coord"])))))
                height = np.int64(np.min(np.diff(np.sort(np.unique(df_sub["Y_coord"])))))
            else:
                width = 10
                height = 10
            
            p.rect(
                x="X_coord",
                y="Y_coord",
                width=width,
                height=height,
                source=source,
                line_color='black',
                line_width = 0.5,
                fill_color=transform(parameter, mapper))

        
        p.text(x="X_coord", y="Y_coord",x_offset = -x_offset,y_offset = y_offset-2, text=parameter+'_mini',source=source,
              text_font_size = {'value': '12px'},text_color='black')

        Hover_p = [("Part number", "@Part_number"),]
        p.add_tools(HoverTool(tooltips=Hover_p))


#         ticker = SingleIntervalTicker(interval=1, num_minor_ticks=0)
#         xaxis = LinearAxis(ticker=ticker)
#         yaxis = LinearAxis(ticker=ticker)
#         p.add_layout(xaxis,"below")
#         p.add_layout(yaxis,"left")




        #Add all plots to parameter
        plots.append(p)
    return plots

In [5]:
def get_html_formatter(my_col):
    template = """
        <div style="background:<%= 
            (function colorfromint(){
                if(result_col == true){
                    return('#8a9f42')}
                else if (result_col == false)
                    {return('#f14e08')}
                }()) %>; 
            color: black"> 
        <%= value %>
        </div>
    """.replace('result_col',my_col)

    return HTMLTemplateFormatter(template=template)



#Create Homogeneity plots
def homogeneity(data,parameters,Colourmap_range,Visual):


                
                
    #If more layers, split based on layer            
    if 'Print_ID_layer' in data.columns:
        Jobs = np.unique(data['Print_ID_layer'])
    else:
        Jobs = np.unique(data['Print_ID'])

    #Prepare collection parameters
    Plots=[]
    tabs1 = []
    job = []
    
    #Convert matplotlib colorbar
    m_coolwarm_rgb = (255 * cm.bwr(range(256))).astype('int')
    coolwarm_palette = [RGB(*tuple(rgb)).to_hex() for rgb in m_coolwarm_rgb]


    #Make mean heatmaps 
    plots = []
    method = 'Mean'
    plots_mean = heatmaps(data,parameters,job,Colourmap_range,method,coolwarm_palette,plots,Visual)

    sec_plot_mean = column(*plots_mean)
    

    tab_mean = Panel(child=sec_plot_mean, title = method)
    tabs1.append(tab_mean)
    
    
    #Make std heatmaps
    plots = []
    method = 'Std'
    plots_std = heatmaps(data,parameters,job,Colourmap_range,method,coolwarm_palette,plots,Visual)

    sec_plot_std = column(*plots_std)


    tab_std = Panel(child=sec_plot_std, title = method)
    tabs1.append(tab_std)



    #Make job based heatmaps 
    for job in Jobs:
        plots = []


        method = 'Original'
        plots = heatmaps(data,parameters,job,Colourmap_range,method,coolwarm_palette,plots,Visual)
        
        #Layout 
        sec_plot = column(*plots)

        tab = Panel(child=sec_plot, title = job)
        tabs1.append(tab)


    #Collect plots. Make two coloumns of tabs for easier comparison 
    Plot = Tabs(tabs=tabs1)
    Plot_copy = Tabs(tabs=tabs1)
    Plots = child=row(Plot, Plot_copy)
    
    
    return Plots


In [6]:
#Box and mean plot 
def Boxplot(Data,parameter,category):
    # generate some synthetic time series for different categories
    
    
    cat = Data[category]
    cats = list(np.unique(cat))
    ### ----- Use this if catagory is a date ----- ###
    #cats = list(np.unique([cats.strftime("%m/%d") for cats in cat]))

    df_box = Data
    df_box['score'] = Data[parameter]


    # find the mean, quartiles and IQR for each category
    groups = df_box[['score',category]].groupby(category)

    q1 = groups.quantile(q=0.25)
    q2 = groups.quantile(q=0.5)
    q3 = groups.quantile(q=0.75)
    iqr = q3 - q1
    upper = q3 + 1.5*iqr
    lower = q1 - 1.5*iqr
    
    mean = groups.mean(numeric_only=True)


    # find the outliers for each category
    def outliers(group):
        cat = group.name
        return group[(group.score > upper.loc[cat]['score']) | (group.score < lower.loc[cat]['score'])]['score']
    out = groups.apply(outliers).dropna()
    

    # prepare outlier data for plotting, we need coordinates for every outlier.
    
    if len(cats)>1:
        if not out.empty:
            outx = list(out.index.get_level_values(0))
            #outx = [outxx.strftime("%m/%d") for outxx in outx]
            outy = list(out.values)
            outindex = list(out.index.get_level_values(1))
            outpart = Data['Part_number'][outindex]
                


    # if no outliers, shrink lengths of stems to be no longer than the minimums or maximums
    qmin = groups.quantile(q=0.00)
    qmax = groups.quantile(q=1.00)
    upper.score = [min([x,y]) for (x,y) in zip(list(qmax.loc[:,'score']),upper.score)]
    lower.score = [max([x,y]) for (x,y) in zip(list(qmin.loc[:,'score']),lower.score)]

    #Color for boxes
    fill_color = []
    for job in np.unique(cat):
        mask = job == Data[category]
        
        fill_color.append('green')
        
        #If daraframe has info on whether part is accepted or not
        #if np.sum(df_box['Accepted_'+code][mask]!='Yes')!=0:
        #    fill_color.append('red')
        #else:
        #    fill_color.append('green')
            
    #Make data frame to enable hover tool    
    vbar = np.ones(len(cats))*0.7
    d = {"cats": cats, "fill_color": fill_color, "q1score": q1.score, "q2score": q2.score, "q3score": q3.score, "upperscore": upper.score, "lowerscore": lower.score, "meanscore": mean.score}
    box_data = pd.DataFrame(d).reset_index() 
    box_data = ColumnDataSource(data=box_data)

    #Make boxplot
    p = figure(x_range=cats,width=800, height=350, x_axis_label = category, y_axis_label = 'Result value')

    # stems
    p.segment("cats", "upperscore", "cats", "q3score", source = box_data, line_color="black")
    p.segment("cats", "lowerscore", "cats", "q1score", source = box_data, line_color="black")

    # boxes
    p.vbar(x="cats", top="q3score", bottom="q2score", width=0.7, source = box_data, fill_color="fill_color", line_color="black")
    p.vbar(x="cats", top="q1score", bottom="q2score", width=0.7, source = box_data, fill_color="fill_color", line_color="black")

    # whiskers (almost-0 height rects simpler than segments)
    p.rect("cats", "lowerscore", 0.2, 0.00001, source = box_data, line_color="black")
    p.rect("cats", "upperscore", 0.2, 0.00001, source = box_data, line_color="black")
    
    #Add mean and standard deveation from acceptn
    #p.line([cats[0], cats[-1]], [Acceptance_values['Mean_'+code], Acceptance_values['Mean_'+code]], line_color = 'black')
    #p.line([cats[0], cats[-1]], [Acceptance_values['Mean_'+code]+Acceptance_values['STD_'+code], Acceptance_values['Mean_'+code]+Acceptance_values['STD_'+code]], line_color = 'black',line_dash = 'dashed')
    #p.line([cats[0], cats[-1]], [Acceptance_values['Mean_'+code]-Acceptance_values['STD_'+code], Acceptance_values['Mean_'+code]-Acceptance_values['STD_'+code]], line_color = 'black',line_dash = 'dashed')
 

    # outliers
    if len(cats)>1:
        if not out.empty:

            outdf = pd.DataFrame({"outx": outx,"outy": outy, "outpart": outpart}).reset_index() 
            p.scatter("outx", "outy", source=outdf, size=16, color="darkorange",fill_alpha=0.6)

    #p.xaxis.major_label_text_font_size="16px"
    #p.yaxis.major_label_text_font_size="16px"
    

    #Add hover tool to boxplot
    Hover_p = [
    ("Median", "@q2score"),
    ("Upper quantile", "@q3score"),
    ("Upper quantile", "@q1score"),
    ("Part number", "@outpart"),
    ]
    p.add_tools(HoverTool(tooltips=Hover_p))
    
    p.add_layout(Title(text=Data['Version'].iloc[-1], align="right",text_color ='grey'), "right")


    return p

In [ ]:
#Values to calculate CpK values
c4=[0.7979,0.8862,0.9213,0.94,0.9515,0.9594,0.965,0.9693,0.9727,0.9754,0.9776,0.9794,0.981,0.9823,0.9835,0.9845,0.9854,0.9862,0.9869,0.9876,0.9882,0.9887,0.9892,0.9896]

In [2]:
# Capability report 
def Capability(df,parameters,Tolerances,category):
    

    
    Jobs = np.unique(df['Print_ID'])
    
    #df preparation 
    df_mean = pd.DataFrame({})
    
    tabs1 = []

    data=df.copy()
    
    
    

    for parameter in parameters:
        df_mean_mini = pd.DataFrame({})
        
        i=0
        plots = []

        colors = itertools.cycle(viridis(len(Jobs))) 

        #Calulate mean and std for full data set
        Overall_mean = np.mean(data[parameter])
        Overall_std = np.std(data[parameter])
        
        #Prep repeatability column
        data[parameter+'_repeatability'] = -9999
        
        
        for part in np.unique(data['Part_number']):
            
            part_mask = data['Part_number'] == part

            
            #Calculate repeatability as result value shifted by part mean
            data[parameter+'_repeatability'][part_mask] = data[parameter][part_mask] - np.mean(data[parameter][part_mask])

        # Repeatability based on all jobs
        Repeatability = np.std(data[parameter+'_repeatability'])

        #Plot of all jobs consecutively 
        p = figure(
            plot_width=1000,
            plot_height=350,
            x_axis_label='Part Number + shift',
            y_axis_label='Result_value',
            title=parameter +'. Overall mean +/- std: '+ str(np.round(Overall_mean ,3)) +'+/-' + str(np.round(Overall_std,3))
            )
        p.add_layout(Title(text=data['Version'].iloc[-1], align="right",text_color ='grey'),"right")
        
        #Part mean
        ppp = figure(
            plot_width=1000,
            plot_height=350,
            x_axis_label='Part Number',
            y_axis_label='Mean result value',
            title='Part mean'
            )
        ppp.add_layout(Title(text=data['Version'].iloc[-1], align="right",text_color ='grey'),"right")

        #Repeatability
        pp = figure(
            plot_width=1000,
            plot_height=350,
            x_axis_label='Part Number',
            y_axis_label='Difference from part mean',
            title='Repeatability: '+ str(np.round(Repeatability ,3))
            )
        pp.add_layout(Title(text=data['Version'].iloc[-1], align="right",text_color ='grey'),"right")
        
        #Histogram
        pppp = figure(
            plot_width=1000,
            plot_height=350,
            x_axis_label='Result Value',
            y_axis_label='Count (scaled)',
            #title='Repeatability: '+ str(np.round(Repeatability ,3))
            )
        pppp.add_layout(Title(text=data['Version'].iloc[-1], align="right",text_color ='grey'),"right")


        for job in Jobs:
            
            color=next(colors)

            #define subset based on job
            df_sub = data[data['Print_ID']==job]
            #Shift part number
            df_sub['Part_number_scaled'] = df_sub['Part_number'] + np.max(df_sub['Part_number'])*i
            i+=1


            #Mean and std of job specific subset
            mean = np.mean(df_sub[parameter])
            std = np.std(df_sub[parameter])


            df_sub[parameter+'_mean'] = mean
            df_sub[parameter+'_std'] = std
            df_sub[parameter+'_std_upper'] = mean+std
            df_sub[parameter+'_std_lower'] = mean-std
            


            #Plot consecutive jobs
            p.scatter('Part_number_scaled',parameter,color=color,source=df_sub,legend_label=job)            
            p.line('Part_number_scaled',parameter+'_mean',source=df_sub,color='black')
            p.line('Part_number_scaled',parameter+'_std_upper',source=df_sub,color='black',line_dash='dashed')
            p.line('Part_number_scaled',parameter+'_std_lower',source=df_sub,color='black',line_dash='dashed')
            
            
            
            #Add tolerance parameters if they exist 
            if parameter+'_USL' in Tolerances:
                df_out = df_sub[(df_sub[parameter]>Tolerances[parameter+'_USL'].values[0]) | (df_sub[parameter]<Tolerances[parameter+'_LSL'].values[0])]
                p.scatter('Part_number_scaled',parameter,color='red',source=df_out)
                p.line( [np.min(df_sub['Part_number_scaled']),np.max(df_sub['Part_number_scaled'])], [Tolerances[parameter+'_USL'],Tolerances[parameter+'_USL']],color='gray'  )
                p.line( [np.min(df_sub['Part_number_scaled']),np.max(df_sub['Part_number_scaled'])], [Tolerances[parameter+'_LSL'],Tolerances[parameter+'_LSL']],color='gray'  )

                
            # Add hover toll 
            parameter_name = '@'+ parameter
            Hover_p = [("Part number", "@Part_number"),("Print ID", "@Print_ID"),(parameter,parameter_name)]
            p.add_tools(HoverTool(tooltips=Hover_p))

            #Add legend 
            p.legend.click_policy="hide"
            plots.append(p)
            
            
            df_mini = df_sub.head(1)
            
         
        #Prep for capability graphs 
        Category = np.unique(df[category])    
        colors = itertools.cycle(viridis(len(Category))) 
        
        df_control = pd.DataFrame({})
        
        
        # Xbar chart for evolution of mean value
        xbar_chart = figure(
        plot_width=1000,
        plot_height=350,
        x_axis_label=category,
        y_axis_label='Mean',
        x_range=Category,
        #title='Repeatability: '+ str(np.round(Repeatability ,3))
        )
        xbar_chart.add_layout(Title(text=data['Version'].iloc[-1], align="right",text_color ='grey'),"right")
        
        # S chart for evolution of std value
        s_chart = figure(
        plot_width=1000,
        plot_height=350,
        x_axis_label=category,
        y_axis_label='Std',
        x_range=Category,
        #title='Repeatability: '+ str(np.round(Repeatability ,3))
        )
        s_chart.add_layout(Title(text=data['Version'].iloc[-1], align="right",text_color ='grey'),"right")
        
        x_range = Range1d(start = -1, end = 5)
        y_range = Range1d(start = -2.2, end = 2.2)
        cpk_plot = figure(
            plot_width=400, 
            plot_height=300, 
            x_axis_type=None, 
            y_axis_type=None, 
            x_range=x_range, 
            y_range=y_range,
            toolbar_location= None)
        cpk_plot.add_layout(Title(text=data['Version'].iloc[-1], align="right",text_color ='grey'), "right")
        
        for job in Category:
            color=next(colors)
            
            df_sub2 = data[data[category]==job]
            
            
            #Histograms
            if not df_sub2[parameter].dropna().empty:
                
                hist, edges = np.histogram(df_sub2[parameter].dropna(), density=True, bins=15)
    
    
                #Anderson-Darling test to check for normality at 95% confidence 
                result = (anderson(df_sub2[parameter].dropna(), dist='norm'))
                AD_test = result.statistic < result.critical_values[2]

                if np.isnan(result.statistic):
                    AD_test = 'ERROR: Contains nan'
                    line = '..'
                else:
                    if AD_test == True:
                        AD_test = 'Normal'
                        line = 'solid'
                    else:
                        AD_test = 'Not normal'
                        line = 'dashed'
            
                # Calculate gaussian fit 
                x = np.linspace(np.min(df_sub2[parameter]), np.max(df_sub2[parameter]), 100)

                mu = np.mean(df_sub2[parameter])
                sigma = np.std(df_sub2[parameter])
                
                df_sub2[parameter+'_mean'] = mu
                df_sub2[parameter+'_std'] = sigma
                

                pdf = 1/(sigma * np.sqrt(2*np.pi)) * np.exp(-(x-mu)**2 / (2*sigma**2))
                
                df_plot = pd.DataFrame({'x': x, 'pdf':pdf})

                top = np.max(pdf)
                
                df_plot['Mean'] = mu
                df_plot['Top'] = top
                df_plot['Print_ID'] = job
                df_plot['Std'] = std

                
                #Plot histogram 
                pppp_legend = job+', '+AD_test+'. Mean '+str(np.round(mu,3))+' +/- '+str(np.round(sigma,3))
                pppp.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],fill_color=color, line_color="white",
                          alpha=0.5,legend_label=pppp_legend)
                pppp.line('x','pdf',line_dash=line,legend_label=pppp_legend,color=color,source=df_plot)


                df_control = pd.concat([ df_control,pd.DataFrame({'Category': [job], 'Mean': [mu], 'Std' : [sigma]}) ])
    
                df_new = pd.DataFrame({'Category': [job], parameter: [str(np.round(mu,3))+'(std:'+str(np.round(sigma,3))+')']})
                
                if df_mean_mini.empty:
                    df_mean_mini = df_new
                else:
                    df_mean_mini = pd.concat([ df_mean_mini,df_new])
                    
        df_new = pd.DataFrame({'Category': ['All prints'], parameter: [str(np.round(Overall_mean,3))+'(std:'+str(np.round(Overall_std,3))+')']})
        if df_mean_mini.empty:
            df_mean_mini = df_new
        else:
            df_mean_mini = pd.concat([ df_mean_mini,df_new])


        if df_mean.empty:
            df_mean = df_mean_mini 
        else:
            df_mean = df_mean.merge(df_mean_mini, on = 'Category')
            
            
        # Add legend 
        leg = p.legend[0]
        p.add_layout(leg,'right')
        
        
        # Add legend 
        leg = pppp.legend[0]
        pppp.add_layout(leg,'right')
        pppp.legend.click_policy="hide"

        
        #Plot repetability data 
        pp.scatter('Part_number',parameter+'_repeatability',color='Peru',source=data)

        #Add hover tool to repeatability plot
        repeatability_name = '@'+parameter+'_repeatability'
        Hover_pp = [("Part number", "@Part_number"),('Print_ID','@Print_ID')
                    ,(parameter,parameter_name),('Difference from mean',repeatability_name)]
        pp.add_tools(HoverTool(tooltips=Hover_pp))
        

        #Plot part mean
        IDs = data[['X_coord','Y_coord','Print_ID','Print_ID_layer','Print_ID_galvo']]
        data_dropped = data.drop(columns=['X_coord','Y_coord','Print_ID','Print_ID_layer','Print_ID_galvo'])
        df_part_mean = data_dropped.groupby('Part_number').mean(numeric_only=True)
        df_part_std = data_dropped.groupby('Part_number').std()
        
        df_part_mean = df_part_mean.reset_index()
        df_part_mean = df_part_mean.merge(IDs, left_index=True, right_index=True)
        
        df_part_std = df_part_std.reset_index()
        df_part_std = df_part_std.merge(IDs, left_index=True, right_index=True)
        
        ppp.scatter('Part_number',parameter,color='Navy',source=df_part_mean)
        
        #If more than one job plot std as errorvbars on part mean value 
        for i in df_part_mean.index:

            if (np.isnan(df_part_mean[parameter][i])==False) & (np.isnan(df_part_std[parameter][i])==False):
                ppp.line([df_part_mean['Part_number'][i],df_part_mean['Part_number'][i]],
                         [df_part_mean[parameter][i]-df_part_std[parameter][i],
                          df_part_mean[parameter][i]+df_part_std[parameter][i]],
                          color='black')

        #Add hover tool 
        parameter_name = '@'+parameter
        Hover_ppp = [("Part number", "@Part_number"),(parameter,parameter_name)]
        ppp.add_tools(HoverTool(tooltips=Hover_ppp))



        #Box plot
        boxplot = Boxplot(data,parameter,category)
        
        
        #Control charts
        df_control = df_control.reset_index(drop=True)
        xbar_chart.line('Category','Mean',source=df_control,color='blue')
        xbar_chart.scatter('Category','Mean',source=df_control,color='blue')
        
        Hover_xbar_chart = [("Category", "@Category"),('Mean','@Mean')]
        xbar_chart.add_tools(HoverTool(tooltips=Hover_xbar_chart))
        
        
        s_chart.line('Category','Std',source=df_control,color='Orange')
        s_chart.scatter('Category','Std',source=df_control,color='Orange')
        
        Hover_s_chart = [("Category", "@Category"),('Std','@Std')]
        s_chart.add_tools(HoverTool(tooltips=Hover_s_chart))
        
        
        xcoor = [0,0,2,2,0,0,2,2]
        ycoor = [1,0,1,0,-1,-2,-1,-2]
        
        #If tolerance is given add control chart calculations
        if parameter+'_USL' in Tolerances:
            
            
            pppp.line([Tolerances[parameter+'_LSL'],Tolerances[parameter+'_LSL']],[0,top],color='black')
            pppp.line([Tolerances[parameter+'_USL'],Tolerances[parameter+'_USL']],[0,top],color='black')
            
            
            xbar_chart.line([0,len(df_control['Category'])],[Tolerances[parameter+'_LSL'],Tolerances[parameter+'_LSL']],color='black')
            xbar_chart.line([0,len(df_control['Category'])],[Tolerances[parameter+'_USL'],Tolerances[parameter+'_USL']],color='black')

            
            
            #Calculate CpK, Cp, Pp, PpK 
            
            # Scale values based on bias correction constart c4 value in accordance with 
            # https://blog.minitab.com/en/marilyn-wheatleys-blog/how-cpk-and-ppk-are-calculated2c-part-2
            # and https://web.mit.edu/2.810/www/files/readings/ControlChartConstantsAndFormulae.pdf
            c4_index = len(Category)-2
            if len(Category) < 2:
                c4_index = 0
            elif (len(Category) >= 15) & len(Category) < 25:
                c4_index = -2
            elif len(Category) >= 25:
                c4_index = -1
            
            c4_select = c4[c4_index]
            
            xbar = Overall_mean
            sigma = Overall_std
            usl = Tolerances[parameter+'_USL']
            lsl = Tolerances[parameter+'_LSL']

            sigma_hat = np.mean(df_control['Std'])/c4_select

            pp_ = (usl-lsl) / (6 * sigma)
            pp_s = (pp_[0].round(3))
            ppk = np.min( [usl-xbar,xbar-lsl] ) /(3 * sigma)
            ppk_s = (ppk.round(2))            

            cp = (usl-lsl) / (6 * sigma_hat)
            cp_s = (cp[0].round(3))
            cpk = np.min( [usl-xbar,xbar-lsl] ) /(3 * sigma_hat)
            cpk_s = (cpk.round(2))

             
            Name = ['Cp', cp_s, 'Cpk', cpk_s, 'Pp', pp_s, 'Ppk', ppk_s]
        else:
            Name = ['Cp', '', 'Cpk', '', 'Pp', '', 'Ppk', '']


        d = {"xcoor": xcoor, "ycoor": ycoor, "Name": Name}
        cpk_source = pd.DataFrame(d).reset_index() 


        cpk_plot.text(x="xcoor", y="ycoor", text="Name", source=cpk_source, text_font_size = {'value': '35px'})
        
        

        #Plot layouts
        sec_plot = column([p,pp,ppp,boxplot,pppp,xbar_chart,s_chart,cpk_plot])

        tab = Panel(child=sec_plot, title = parameter)
        tabs1.append(tab)
        
    #Summary table. Transpose for easier read
    df_mean_transposed = df_mean.T

    df_mean_transposed.columns = df_mean_transposed.iloc[0]

    df_mean_transposed = df_mean_transposed.drop(df_mean_transposed.index[0])
    df_mean_transposed['Parameter'] = df_mean_transposed.index
    
    df_mean_transposed = df_mean_transposed.reset_index()
    
    df_mean_transposed =  df_mean_transposed.drop(columns=['index'])

    
    temp_cols=df_mean_transposed.columns.tolist()
    new_cols=temp_cols[-1:] + temp_cols[:-1]
    df_mean_transposed=df_mean_transposed[new_cols]
    
    #Plot table
    Columns = [TableColumn(field=Ci, title=Ci) for Ci in df_mean_transposed.columns] # bokeh columns
    data_table = DataTable(columns=Columns, source=ColumnDataSource(df_mean_transposed))
    
    
    table_tab = Panel(child=data_table, title = 'Summary')
    tabs1.insert(0,table_tab)

    Plots = Tabs(tabs=tabs1)
                
    return Plots

In [1]:
#Run calculations and plots
def Run(Path,Data_folder,Save_folder,Parameters = [],Visual='Rectangles',split=True,Tolerances=[],Colourmap_range=[],category = 'Print_ID',PartID_loc='First'):
    
    #Make file
    df_full = make_file(Path,Data_folder,Save_folder,Version,PartID_loc)
        

    #Define parameters to plot (or the ones to not)
    if len(Parameters) != 0:
        parameters = Parameters
    else:
        parameters = df_full.columns.drop(['Part_number','Cube','Date','Print_ID','X_coord','Y_coord','Layer', 'Print_ID_layer','Print_ID_galvo','Machine','Version'])
    
    for cols in ['Galvo','Exposure_order','Z_coord']:
        if cols in parameters:
            parameters = parameters[parameters != cols]
    

    #Plot homogeneity 
    if split:
        tabs = []
        for machine in np.unique(df_full['Machine']):
            
            sec_plot = homogeneity(df_full[df_full['Machine']==machine],parameters,Colourmap_range,Visual)

            tab = Panel(child=sec_plot, title = str(machine))
            tabs.append(tab)
        Plots = Tabs(tabs=tabs)

    else:
        Plots = homogeneity(df_full,parameters,Colourmap_range,Visual)

    output_file(filename=Save_folder+"/Heatmaps.html", title="Heatmaps")
    show(Plots)
    
  
    #plot Capability plots      
    if split:
        tabs = []
        for machine in np.unique(df_full['Machine']):
            sec_plot = Capability(df_full[df_full['Machine']==machine],parameters,Tolerances,category)

            tab = Panel(child=sec_plot, title = str(machine))
            tabs.append(tab)
        Plots = Tabs(tabs=tabs)
    else:
        Plots = Capability(df_full,parameters,Tolerances,category)    
    
        
        
    output_file(filename=Save_folder+"/Capability.html", title="Capability")
    show(Plots)
    